# Landscape or Cityscape?

The idea behind this notebook is to test out various CNNs to determine which networks are best suited to determining if an picture is an image of a city or an image of nature. My data sources for this experiment are the subreddits r/earthporn and r/cityporn. I chose these subreddits as they are focused and fairly strictly moderated. Each contains only aestetically pleasing pictures of landscapes and cityscapes respectively. I took all the directly linked images from these subreddits since 1/1/17 that have over a certain threshold of upvotes, which gave me a few thousand images that are almost entirely either a picture of a natural landscape or a picture of a cityscape

I will use keras' implementation of VGG16, ResNet50 and InceptionV3 to extract features from the images, and then I will contruct a CNN of my own design and compare the results

In [ ]:
#general imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [113]:
#keras specific imports
from keras import Model, Input
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from keras.applications import VGG16,vgg16
from keras.applications import ResNet50, resnet50
from keras.applications import InceptionV3, inception_v3

In [32]:
def prep_for_vgg(path):
    img = image.load_img(path, target_size = (224, 224) )
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = vgg16.preprocess_input(x, mode='tf')
    return x

In [73]:
VGG = VGG16(weights = 'imagenet', include_top = False, input_shape= (224,224,3))

In [138]:
def create_model(base_layers):
    X = base_layers.output
    X = Flatten()(X)
    X = Dense(100, activation = 'relu',name = '1')(X)
    predictions = Dense(2 , activation = 'softmax',name = '2')(X)
    print(predictions.shape)
    model = Model(inputs = base_layers.inputs, outputs = predictions)
    for layer in base_layers.layers:
        layer.trainable = False
    return model
    

In [90]:
data_gen = ImageDataGenerator(shear_range= 0.2,
                                    zoom_range= 0.2,
                                    horizontal_flip= True,
                                    width_shift_range=0.2,
                                    height_shift_range = 0.2,
                                    preprocessing_function= (lambda x: x/127.5 - 1))                                           

train_generator = data_gen.flow_from_directory('./images', target_size= (224,224), class_mode= 'categorical')

vgg_model = create_model(VGG)

vgg_model.compile(optimizer='rmsprop',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
vgg_model.fit_generator(data_generator, epochs=10, steps_per_epoch= 500,
                        use_multiprocessing=True)